In [ ]:
from pynput import keyboard, mouse
import time, csv, os, signal, sys
from datetime import datetime
import logging
# === Ensure data folder exists ===
os.makedirs("../data", exist_ok=True)

# === Setup Logging ===
logging.basicConfig(filename='../data/cua.log', level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Starting CUA data capture service.")

# === File paths ===
keyboard_file = "../data/keyboard_raw.csv"
mouse_file = "../data/mouse_raw.csv"

# === Open files in append mode ===
kb_file = open(keyboard_file, "a", newline="")
ms_file = open(mouse_file, "a", newline="")

kb_writer = csv.writer(kb_file)
ms_writer = csv.writer(ms_file)

# === Optional: write headers if file is empty ===
if os.stat(keyboard_file).st_size == 0:
    kb_writer.writerow(["event_type", "key", "timestamp", "datetime"])

if os.stat(mouse_file).st_size == 0:
    ms_writer.writerow(["event_type", "x", "y", "timestamp", "datetime"])

# === Mouse listener needs to be globally accessible ===
ms_listener = None

# === Keyboard Handlers ===
def on_press(key):
    try:
        kb_writer.writerow(["press", key.char, time.time(), datetime.now()])
    except AttributeError:
        kb_writer.writerow(["press", str(key), time.time(), datetime.now()])
    kb_file.flush()

def on_release(key):
    try:
        kb_writer.writerow(["release", key.char, time.time(), datetime.now()])
    except AttributeError:
        kb_writer.writerow(["release", str(key), time.time(), datetime.now()])
    kb_file.flush()

    if key == keyboard.Key.esc:
        stop_listeners()

# === Mouse Handlers ===
def on_move(x, y):
    try:
        ms_writer.writerow(["move", x, y, time.time(), datetime.now()])
        ms_file.flush()
    except ValueError:
        pass

def on_click(x, y, button, pressed):
    try:
        event = "press" if pressed else "release"
        ms_writer.writerow([event, x, y, time.time(), datetime.now()])
        ms_file.flush()
    except ValueError:
        pass

# === Cleanup Handler ===
def stop_listeners(signum=None, frame=None):
    print("🛑 Stopping capture and closing files.")
    if ms_listener: ms_listener.stop()
    kb_file.close()
    ms_file.close()
    sys.exit(0)

# === Signal handler for Ctrl+C ===
signal.signal(signal.SIGINT, stop_listeners)

from pynput import keyboard, mouse
import time, csv, os, signal, sys
from datetime import datetime
import logging

# === Setup Logging ===
logging.basicConfig(filename='../data/cua.log', level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Starting CUA data capture service.")

# === Ensure data folder exists ===
os.makedirs("../data", exist_ok=True)

# === File paths ===
keyboard_file = "../data/keyboard_raw.csv"
mouse_file = "../data/mouse_raw.csv"

# === Open files in append mode ===
kb_file = open(keyboard_file, "a", newline="")
ms_file = open(mouse_file, "a", newline="")

kb_writer = csv.writer(kb_file)
ms_writer = csv.writer(ms_file)

# === Optional: write headers if file is empty ===
if os.stat(keyboard_file).st_size == 0:
    kb_writer.writerow(["event_type", "key", "timestamp", "datetime"])

if os.stat(mouse_file).st_size == 0:
    ms_writer.writerow(["event_type", "x", "y", "timestamp", "datetime"])

# === Mouse listener needs to be globally accessible ===
ms_listener = None

# === Keyboard Handlers ===
def on_press(key):
    try:
        kb_writer.writerow(["press", key.char, time.time(), datetime.now()])
    except AttributeError:
        kb_writer.writerow(["press", str(key), time.time(), datetime.now()])
    kb_file.flush()
    logging.info(f"Key press: {key}")

def on_release(key):
    try:
        kb_writer.writerow(["release", key.char, time.time(), datetime.now()])
    except AttributeError:
        kb_writer.writerow(["release", str(key), time.time(), datetime.now()])
    kb_file.flush()
    logging.info(f"Key release: {key}")
    if key == keyboard.Key.esc:
        stop_listeners()

# === Mouse Handlers ===
def on_move(x, y):
    try:
        ms_writer.writerow(["move", x, y, time.time(), datetime.now()])
        ms_file.flush()
        logging.info(f"Mouse move: ({x}, {y})")
    except ValueError:
        pass

def on_click(x, y, button, pressed):
    try:
        event = "press" if pressed else "release"
        ms_writer.writerow([event, x, y, time.time(), datetime.now()])
        ms_file.flush()
        logging.info(f"Mouse {event}: ({x}, {y})")
    except ValueError:
        pass

# === Cleanup Handler ===
def stop_listeners(signum=None, frame=None):
    logging.info("Stopping CUA data capture service.")
    if ms_listener: ms_listener.stop()
    kb_file.close()
    ms_file.close()
    sys.exit(0)

# === Signal handler for Ctrl+C ===
signal.signal(signal.SIGINT, stop_listeners)

# === Main Loop for Continuous Operation ===
while True:
    try:
        print("🔴 CUA Capture started — Press ESC or Ctrl+C to stop. Move/click mouse to record events.")
        kb_listener = keyboard.Listener(on_press=on_press, on_release=on_release)
        ms_listener = mouse.Listener(on_move=on_move, on_click=on_click)

        kb_listener.start()
        ms_listener.start()

        kb_listener.join()
    except Exception as e:
        logging.error(f"Error occurred: {e}")
        time.sleep(5)  # Wait before restarting to avoid tight loop

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.
This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


🔴 Capture started — Press ESC to stop. Move/click mouse to record events.
🛑 Stopping capture and closing files.
